In [2]:
pip install elasticsearch

In [3]:
# Data Source “http://files.grouplens.org/datasets/movielens/ml-latest-small.zip”

In [4]:
import csv
from collections import deque
import elasticsearch
from elasticsearch import helpers
from builtins import int


ModuleNotFoundError: No module named 'elasticsearch'

In [ ]:
def readMovies():
    csvfile = open(r"C:\Users\sanjay\Desktop\ml-latest-small\movies.csv",encoding="utf8")
    #Dict reader used to create key value pair similar to JSON
    reader = csv.DictReader( csvfile )

    titleLookup = {}

    for movie in reader:
            #Lookup table to match 
            titleLookup[movie['movieId']] = movie['title']

    return titleLookup

In [ ]:
def readRatings():
    csvfile = open(r'C:\Users\sanjay\Desktop\ml-latest-small\ratings.csv',encoding="utf8")
    #Calling the previous lookup 
    titleLookup = readMovies()

    reader = csv.DictReader( csvfile )
    for line in reader:
        rating = {}
        rating['user_id'] = int(line['userId'])
        rating['movie_id'] = int(line['movieId'])
        rating['title'] = titleLookup[line['movieId']]#Make a look up similar to JOIN in SQL
        rating['rating'] = float(line['rating'])
        rating['timestamp'] = int(line['timestamp'])
        yield rating #Creating a Generator function

In [ ]:
es = elasticsearch.Elasticsearch()

#Delete the indices if previously created
es.indices.delete(index="ratings",ignore=404)
#using the functions we can do a bulk import and call the readRatings functions and name the index as Ratings 
deque(helpers.parallel_bulk(es,readRatings(),index="ratings"), maxlen=0)
es.indices.refresh()